In [ ]:
from milvus import Milvus, IndexType, MetricType, Status

In [ ]:
milvus = Milvus(host='localhost', port='19530')

In [ ]:
milvus.create_collection({
    'collection_name': 'memes_small',
    'dimension': 512
    })

Status(code=0, message='Create collection successfully!')

In [ ]:
import torch
import torchvision
import clip
import numpy as np

from tqdm import tqdm
from pathlib import Path
from IPython.display import Image, display
from PIL import Image as Img, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# set torch 🔦 device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# load CLIP 📎 model
model, preprocess = clip.load("ViT-B/32", device)
logit_scale = model.logit_scale.exp()
device

device(type='cuda')

In [ ]:
path = Path('images/')

In [ ]:
def preproc_images(path):
    '''Batch process images with CLIP and return their feature embeddings'''
    image_features = torch.tensor(()).to(device)
    with torch.no_grad():
        imagefiles=torchvision.datasets.ImageFolder(root=path, transform=preprocess)
        img_loader=torch.utils.data.DataLoader(imagefiles, batch_size=128, shuffle=False, num_workers=4)
        for images, labels in tqdm(img_loader):
            batch_features = model.encode_image(images)
            image_features = torch.cat((image_features, batch_features)).to(device)

    image_names = [Path(f[0]) for f in imagefiles.imgs]
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)

    return(image_names, image_features)


In [ ]:
image_names, image_features = preproc_images(path)

100%|██████████| 1/1 [00:02<00:00,  2.45s/it]


After a lot of research, I found I don't want to use Milvus because it can't operate on ARM devices
